# Projeto de Bases de Dados - Parte 3

### Docente Responsável

- Prof. André Patrício
- Prof. Jorge Oliveira

### Grupo 47 - Turno BD2L17
<dl>
    <dt>10 horas (33.3%)</dt>
    <dd>ist102927 Henrique Soares</dd>
    <dt>10 horas (33.3%)</dt>
    <dd>ist103606 Henrique Silva</dd>
    <dt>10 horas (33.3%)</dt>
    <dd>ist103624 Afonso Azaruja</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio.

In [ ]:
%%sql
-- populate.sql

INSERT into customer values (01, 'Luis Nogueira', 'luisnogueira@gmail.com', 968754123 ,'Rua de Cima 1, 4480-110 Árvore');
INSERT into customer values (02, 'Vitoria Silva', 'vitoriasilva1@gmail.com' ,917654321 ,'Rua das Orquídeas 59, 2690-408 Santa Iria de Azóia');
INSERT into customer values (03, 'Rita Avestruz', 'ritaavestruz@gmail.com' , 936874521 ,'Bairro Novo da Sardoa 1, 7670-132 Garvão');
INSERT into customer values (04, 'Alberto Bito', 'albertobito@hotmail.pt' , 966152489 ,'Rua Florbela Espanca 11, 7170-072 Redondo');
INSERT into customer values (05, 'Pedro Baril', 'pedrobarilz@hotmail.pt' , 987654321 ,'Rua Mariano Feio 3, 7900-195 Ferreira do Alentejo');

INSERT into orders values (00001, 04, '2023-01-10');
INSERT into orders values (00002, 04, '2022-05-11');
INSERT into orders values (00003, 01, '2019-03-01');
INSERT into orders values (00004, 04, '2023-01-23');
INSERT into orders values (00005, 01, '2019-06-22');
INSERT into orders values (00006, 02, '2017-11-25');
INSERT into orders values (00007, 01, '2020-12-06');
INSERT into orders values (00008, 02, '2022-07-10');
INSERT into orders values (00009, 02, '2012-01-01');
INSERT into orders values (00010, 03, '2023-01-05');
INSERT into orders values (00011, 04, '2023-01-11');
INSERT into orders values (00012, 03, '2023-01-07');
INSERT into orders values (00013, 05, '2023-03-18');
INSERT into orders values (00014, 05, '2018-05-26');
INSERT into orders values (00015, 05, '2023-01-17');

INSERT into pay values (00005, 01);
INSERT into pay values (00006, 02);
INSERT into pay values (00007, 01);
INSERT into pay values (00008, 02);
INSERT into pay values (00009, 02);
INSERT into pay values (00010, 03);
INSERT into pay values (00011, 04);
INSERT into pay values (00012, 03);
INSERT into pay values (00013, 05);
INSERT into pay values (00014, 05);
INSERT into pay values (00015, 05);

INSERT into employee values (01, 949103209, '2000-10-02', 'Josefino Antunes');
INSERT into employee values (02, 949091032, '1999-12-01', 'Leonardo Faísca');
INSERT into employee values (03, 109093294, '1998-06-02', 'Roberto Estrada');
INSERT into employee values (04, 291034909, '1990-01-22', 'Maria Costa');
INSERT into employee values (05, 909103294, '2001-03-20', 'Susana Grilo');

INSERT into process values (01, 00005);
INSERT into process values (04, 00002);
INSERT into process values (04, 00007);
INSERT into process values (04, 00008);
INSERT into process values (05, 00009);
INSERT into process values (01, 00010);
INSERT into process values (02, 00011);
INSERT into process values (03, 00006);
INSERT into process values (01, 00013);
INSERT into process values (03, 00014);
INSERT into process values (05, 00015);

INSERT into department values ('Recursos Humanos');
INSERT into department values ('Inventario');
INSERT into department values ('Logistica');
INSERT into department values ('Apoio ao Cliente');
INSERT into department values ('Marketing');

INSERT into workplace values ( 'Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal' , 10.234 , 40.402 );
INSERT into workplace values ( 'Rua do Centro Social 38, 1750-204 Lisboa' , 12.214 , 39.202 );
INSERT into workplace values ( 'Rua do Tomás 15, 4825-134 Lamelas' , 99.235 , 40.402 );
INSERT into workplace values ( 'Rua Artur Costa Baril 108, 7570-190 Grândola' , 95.144 , 42.442 );
INSERT into workplace values ( 'Rua dos Félix 47, 2125-174 Marinhais' , 93.654 , 38.098 );

INSERT into office values ('Rua do Centro Social 38, 1750-204 Lisboa');
INSERT into office values ('Rua dos Félix 47, 2125-174 Marinhais');

INSERT into warehouse values ('Rua do Centro Social 38, 1750-204 Lisboa');
INSERT into warehouse values ('Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal');
INSERT into warehouse values ('Rua do Tomás 15, 4825-134 Lamelas');
INSERT into warehouse values ('Rua Artur Costa Baril 108, 7570-190 Grândola');

INSERT into works values (01, 'Recursos Humanos', 'Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal');
INSERT into works values (01, 'Recursos Humanos', 'Rua do Centro Social 38, 1750-204 Lisboa');
INSERT into works values (02, 'Inventario', 'Rua do Centro Social 38, 1750-204 Lisboa');
INSERT into works values (03, 'Logistica', 'Rua do Tomás 15, 4825-134 Lamelas');
INSERT into works values (04, 'Apoio ao Cliente', 'Rua Artur Costa Baril 108, 7570-190 Grândola');
INSERT into works values (05, 'Marketing', 'Rua Artur Costa Baril 108, 7570-190 Grândola');

INSERT into product values (2654985632, 'vela ecologica', 'Esta vela e perfumada com uma fragrancia floral natural. Uma combinacao de perfume terapeutico de lavanda e ervas aromaticas com a frescura da canfora. Agradavel para todo o ano e ocasioes. Uma combinacao de luz brilhante e perfume envolvente que cria um ambiente exotico que estimula os sentidos.', 15.35);
INSERT into product values (4653215258, 'SWEET DREAMS ARE MADE OF TEA', 'A Tisana Sweat Dreams Are Made of Tea produzida pela Paisagindo.bio., tem propriedades relaxantes e calmantes. E excelente quando bebida quente antes de dormir ou em alturas de maior stress.', 4.10);
INSERT into product values (6352945125, 'TEMPERO MEDITERRANICO 25g', 'Esta mistura da Paisagingo.bio e feita apenas e so com plantas aromaticas, legumes e especiarias desidratadas a baixa temperatura e da melhor qualidade foi pensada para os amantes dos sabores do Sul da Europa e dos beneficios da dieta mediterranica.', 3.90);
INSERT into product values (8659523254, 'ULTRA CLEAN', 'O Ultra Clean e um produto 2 em 1, desengordurante e desinfetante concentrado. Desengordurante biodegradavel, germicida, bactericida e inodoro. O Ultra Clean penetra profundamente para remover as camadas de gordura, sujidade, derrames de oleo, aliado com o seu poder desinfetante. Nao contem nafta, benzina ou solventes clorados.', 50.90);
INSERT into product values (6352145236, 'TOALHITAS DESINFETANTES', 'As toalhitas desinfetantes Lunette sao o teu produto de limpeza para levar, quando nao tens agua por perto. Viagens de carro, casas de banho dos avioes ou dos festivais, ou o cubiculo do escritorio - vais estar totalmente preparada. As toalhitas desinfetantes Lunetteeliminam grande parte dos germes, permitindo que uses o teu copo menstrual em seguranca.', 4.90);
INSERT into product values (4251452152, 'SABONETE ESFOLIANTE MENTA 100g', 'Seja qual for o seu tipo de pele, esfolie, renove e refresque-a com o sabonete esfoliante de menta da Unii a base de oleo essencial de hortela-pimenta. Bastante energetico, este sabonete promove a renovacao celular, removendo impurezas e celulas mortas, deixando a pele como nova!', 70.90);
INSERT into product values (8659625412, 'SABONETE SUAVIZANTE AMENDOAS DOCES & ALFAZEMA 100g', 'O Sabonete de Amendoas Doces & Lavanda da Unii foi desenvolvido para peles sensiveis, corpo e rosto. O oleo de amendoas doces tem uma acao protetora e em conjunto com a alfazema proporciona um efeito calmante na pele, deixando-a com um suave aroma de amendoas doces e alfazema.', 6.90);
INSERT into product values (2635986532, 'SAL - SABONETE DE SAL MARINHO 100g', 'E um sabonete mais alternativo pela quantidade de sal presente no sabao. O sal tem acao bactericida, ajuda a desintoxicar e a eliminar as impurezas e celulas mortas da superficie da pele. Sabonete com o poder revitalizante do sal e a frescura do oleo essencial de hortela-pimenta, que cria uma experiencia unica de banho. Trata-se de uma barra de sal com um poder de limpeza extraordinario.', 6.00);
INSERT into product values (0215124521, 'SO LOIÇA - SABAO PARA A LOICA 180g', 'Sabao para a loica e o seu aliado perfeito para uma lavagem manual eficaz. Elimina facilmente sujidades e gorduras de todo o tipo de loica. Isto com uma espuma fina e um aroma agradavel a limao e alfazema.', 45.90);
INSERT into product values (0231564896, 'PROTETOR SOLAR NATURAL SPF50 100ml', 'O protetor solar natural SPF 50 da Sol de Ibiza foi concebido para uma protecao eficaz contra os raios UVA e UVB, com ingredientes 100% naturais. Formulado com oxido de zinco nao nano, e resistente a agua e nao contem conservantes ou ingredientes toxicos para nos e para o meio ambiente. E adequado para adultos e criancas, para todos os tipos de pele e pode ser utilizado diariamente.', 52.10);
INSERT into product values (2514251425, 'SABONETE DE AZEITE 75g', 'Sabonete Aromas do Valado e feito a base azeite, sem oleos essenciais, aromatizado apenas com hidrolato (agua floral) de esteva.', 4.50);
INSERT into product values (8965478512, 'PASTA DE DENTES NATURAL CARVAO COM FLUOR 75ml', 'Cuidadosamente formulada para prevenir cáries e proteger ativamente contra as cáries e placa bacteriana. O sabor a menta fresca ajuda a prevenir o mau hálito enquanto a infusão de carvão natural ajuda a remover manchas e descolorações.', 4.45);
INSERT into product values (6235632563, 'PASTA DE DENTES PASTILHAS FLUOR 60u', 'As pastilhas sao cuidadosamente formulados para prevenir caries e proteger ativamente contra caries e placas. Livre de SLS e parabenos com sabor natural de menta fresca e 1450 ppm de fluoreto de sodio. Sao Vegan e Cruelty-free.', 4.99);
INSERT into product values (8456321595, 'OLEO PURO ROSA MOSQUETA 50ml', 'O Oleo Puro Biologico Rosa Mosqueta da Unii e a melhor fonte vegetal de omega 3 e omega 6 no combate ao envelhecimento e na regeneracao da pele. Vai deixar a sua pele com um suave aroma de rosa mosqueta. Previne e minimiza estrias e cicatrizes.', 60.23);
INSERT into product values (8888866666, 'LUVA DE MASSAGEM DE LINHO', 'Luva de banho confeccionada em linho. Gracas ao seu formato de luva, a sua utilizacao e extremamente facil. Ideal para a remocao de celulas mortas e ativacao da circulacao.', 10.40);
INSERT into product values (9562849562, 'Sabonete para barba 100g', 'Sabonete a base de azeite com oleos essenciais que ajudam a minimizar o efeito irritativo do barbear acalmando e suavizando a pele.', 6.00);
INSERT into product values (9638527415, 'Oleo de barba 30ml', 'O Oleo de Barba da Healthy Planet e formulado com os melhores oleos essenciais, tal como o Oleo de Semente de Uva e o Oleo de Argao.', 11.50);
INSERT into product values (6546546546, 'Barra de Barbear Power Shave 80g', 'A barra de barbear Power Shave prepara a pele para um barbear mais agradavel, minimizando a irritacao e a vermelhidao.', 9.90);
INSERT into product values (3213213213, 'Conjunto de Sabonete e Pincel', 'Colecao especial para homem e um complemento ao sabonete de barbear.', 18.00);
INSERT into product values (3265149587, 'CONJUNTO DE SABONETE, PINCEL E TACA', 'Colecao especial para homem e um complemento ao sabonete de barbear.', 25.00);
INSERT into product values (9568965895, 'BARBUDO - SABONETE PARA A BARBA 100g', 'Sabonete a base de azeite extra virgem e aromatizado com oleo essencial de geranio, otimo para lavar a barba e rosto.', 6.00);
INSERT into product values (1490242093, 'CHAMPO SOLIDO E SABONETE 2 EM 1 POWER SHOWER 100g', 'Este champo e sabonete 2 em 1 da Shaeco oferece a pratico e simples que qualquer homem procura!', 9.90);
INSERT into product values (5623154251, 'DESODORIZANTE PEDRA DE ALUMEN BIORK', 'O Biork e um desodorizante completamente ecologico, feito de pedra de alumen de potassio com origem na Malasia e revestido de cortica portuguesa reciclavel.', 13.90);
INSERT into product values (5490242093, 'CHAMPO SOLIDO ESFOLIANTE DETOX 75g', 'Este champo esfoliante de limpeza profunda da Unii e perfeito para uma desintoxicacao e purificacao profunda de todos os tipos de cabelo.', 9.90);
INSERT into product values (9490242093, 'GALEGA - SABONETE DE AZEITE DE OLIVEIRA 100g', 'Sabonete de azeite extra virgem sem aroma adicionado.', 6.00);
INSERT into product values (7949024209, 'SABONETE DE CARVAO ATIVADO E PINHO 100g', 'Sabonete a base de azeite extra virgem, carvao ativado e aromatizado com oleo essencial de pinho.', 6.00);
INSERT into product values (3624678687, 'BALSAMO LABIAL DE CANHAMO', 'Balsamo labial  da Suntribe e altamente hidratante e calmante. Fornece os primeiros socorros para labios secos e gretados, gracas ao oleo de semente de canhamo, um oleo radicalmente hidratante.', 6.99);

INSERT into supplier values (987123465, 'Fornecedor dos Fornecimentos 00 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 2654985632, '2020-01-01' );
INSERT into supplier values (987123461, 'Fornecedor dos Fornecimentos 01 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 4653215258, '2020-01-01' );
INSERT into supplier values (987123460, 'Fornecedor dos Fornecimentos 02 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 6352945125, '2020-01-01' );
INSERT into supplier values (987123462, 'Fornecedor dos Fornecimentos 03 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 8659523254, '2020-01-01' );
INSERT into supplier values (987123463, 'Fornecedor dos Fornecimentos 04 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 6352145236, '2020-01-01' );
INSERT into supplier values (987123464, 'Fornecedor dos Fornecimentos 05 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 4251452152, '2020-01-01' );
INSERT into supplier values (987123467, 'Fornecedor dos Fornecimentos 06 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 8659625412, '2020-01-01' );
INSERT into supplier values (987123469, 'Fornecedor dos Fornecimentos 07 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 2635986532, '2020-01-01' );
INSERT into supplier values (987123415, 'Fornecedor dos Fornecimentos 08 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 0215124521, '2020-01-01' );
INSERT into supplier values (987123425, 'Fornecedor dos Fornecimentos 09 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 0231564896, '2020-01-01' );
INSERT into supplier values (987123435, 'Fornecedor dos Fornecimentos 10 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 2514251425, '2020-01-01' );
INSERT into supplier values (987123445, 'Fornecedor dos Fornecimentos 11 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 8965478512, '2020-01-01' );
INSERT into supplier values (987123455, 'Fornecedor dos Fornecimentos 12 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 6235632563, '2020-01-01' );
INSERT into supplier values (987123475, 'Fornecedor dos Fornecimentos 13 Lda.', 'Rua das Industrias 1, 8600-306 Lagos', 8456321595, '2020-01-01' );
INSERT into supplier values (109872386, 'Rei dos Fornecedores 01 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 8888866666, '2021-01-01');
INSERT into supplier values (109872396, 'Rei dos Fornecedores 02 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 9562849562, '2021-01-01');
INSERT into supplier values (109172346, 'Rei dos Fornecedores 03 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 9638527415, '2021-01-01');
INSERT into supplier values (109272346, 'Rei dos Fornecedores 04 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 6546546546, '2021-01-01');
INSERT into supplier values (109372346, 'Rei dos Fornecedores 05 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 3213213213, '2021-01-01');
INSERT into supplier values (109472346, 'Rei dos Fornecedores 06 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 9568965895, '2021-01-01');
INSERT into supplier values (109572346, 'Rei dos Fornecedores 07 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 1490242093, '2021-01-01');
INSERT into supplier values (109672346, 'Rei dos Fornecedores 08 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 5623154251, '2021-01-01');
INSERT into supplier values (109772346, 'Rei dos Fornecedores 09 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 5490242093, '2021-01-01');
INSERT into supplier values (109872346, 'Rei dos Fornecedores 10 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 9490242093, '2021-01-01');
INSERT into supplier values (109972346, 'Rei dos Fornecedores 11 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 7949024209, '2021-01-01');
INSERT into supplier values (100872346, 'Rei dos Fornecedores 12 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 3624678687, '2021-01-01');
INSERT into supplier values (101872346, 'Rei dos Fornecedores 13 Lda.', 'Rua Alberto Caeiro 2, 7000-409 Évora', 3265149587, '2021-01-01');

INSERT into contains values (00001, 8659523254, 2);
INSERT into contains values (00002, 2654985632, 1);
INSERT into contains values (00003, 4653215258, 4);
INSERT into contains values (00004, 1490242093, 5);
INSERT into contains values (00005, 3213213213, 1);
INSERT into contains values (00006, 8965478512, 4);
INSERT into contains values (00007, 6235632563, 1);
INSERT into contains values (00008, 8456321595, 4);
INSERT into contains values (00009, 8888866666, 7);
INSERT into contains values (00010, 9562849562, 2);
INSERT into contains values (00011, 6546546546, 1);
INSERT into contains values (00012, 3213213213, 2);
INSERT into contains values (00013, 3265149587, 3);
INSERT into contains values (00014, 7949024209, 5);
INSERT into contains values (00015, 3624678687, 1);

INSERT into delivery values ('Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal', 987123465);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 987123461);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola',987123460);
INSERT into delivery values ('Rua do Centro Social 38, 1750-204 Lisboa', 987123462);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 987123463);
INSERT into delivery values ('Rua do Centro Social 38, 1750-204 Lisboa', 987123465);
INSERT into delivery values ('Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal', 987123464);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 987123469);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 987123415);
INSERT into delivery values ('Rua do Centro Social 38, 1750-204 Lisboa', 987123425);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 987123435);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 987123445);
INSERT into delivery values ('Rua do Centro Social 38, 1750-204 Lisboa', 987123455);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 987123475);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109872386);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 109872396);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109172346);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 109272346);
INSERT into delivery values ('Rua do Centro Social 38, 1750-204 Lisboa', 109372346);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109872346);
INSERT into delivery values ('Rua Artur Costa Baril 108, 7570-190 Grândola', 109472346);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109572346);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109672346);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109772346);
INSERT into delivery values ('Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal', 109872346);
INSERT into delivery values ('Rua do Tomás 15, 4825-134 Lamelas', 109972346);
INSERT into delivery values ('Rua Simplicio dos Passos Gouveia 27, 9004-565 Funchal', 100872346);


## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql

ALTER TABLE employee ADD CONSTRAINT check_age CHECK (DATE_TRUNC('year', AGE(CURRENT_DATE, bdate)) >= INTERVAL '18 years') --age must be >=18

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION check_address_constraint() RETURNS TRIGGER AS
$$
DECLARE
    office_count INTEGER;
    warehouse_count INTEGER;
BEGIN
    --  check if address exists in office
    SELECT COUNT(*) INTO office_count
    FROM office
    WHERE address = NEW.address;

    -- check if address exists in warehouse
    SELECT COUNT(*) INTO warehouse_count
    FROM warehouse
    WHERE address = NEW.address;

    -- if it exists in both or none
    IF (office_count = warehouse_count) THEN
        RAISE EXCEPTION 'Address must be in warehouse or office but not both';
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


-- Office trigger
CREATE CONSTRAINT TRIGGER check_address_constraint_office
AFTER INSERT OR UPDATE OR DELETE ON office DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_address_constraint();

-- Warehouse trigger
CREATE CONSTRAINT TRIGGER check_address_constraint_warehouse
AFTER INSERT OR UPDATE OR DELETE ON warehouse DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_address_constraint();

-- Workplace trigger
CREATE CONSTRAINT TRIGGER check_address_constraint_workplace
AFTER INSERT OR UPDATE ON workplace DEFERRABLE INITIALLY DEFERRED
FOR EACH ROW
EXECUTE FUNCTION check_address_constraint();

UsageError: Cell magic `%%sql` not found.


(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.


In [ ]:
%%sql

CREATE OR REPLACE FUNCTION check_order_contains() RETURNS TRIGGER AS
$$
  BEGIN
    IF NEW.order_no NOT IN (SELECT order_no FROM contains)
    THEN RAISE EXCEPTION 'Order % must contain some products', NEW.order_no;
    END IF;
    RETURN NEW;
  END;
$$ LANGUAGE plpgsql;

CREATE CONSTRAINT TRIGGER check_order_contains_trigger AFTER INSERT ON orders DEFERRABLE INITIALLY DEFERRED
  FOR EACH ROW EXECUTE FUNCTION check_order_contains();

UsageError: Cell magic `%%sql` not found.


## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?

In [ ]:
%%sql

SELECT cust_no, customer.name FROM customer
  JOIN pay USING (cust_no) JOIN contains USING (order_no) JOIN product USING (SKU)
  GROUP BY cust_no HAVING SUM(contains.qty * product.price) >= ALL(
    SELECT SUM(contains.qty * product.price) FROM pay
    JOIN contains USING (order_no) JOIN product USING (SKU)
    GROUP BY cust_no
  );

UsageError: Cell magic `%%sql` not found.


:2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql

SELECT e.name
FROM employee e
WHERE NOT EXISTS (
  SELECT o.date
  FROM orders o
  WHERE o.date >= DATE '2022-01-01' AND o.date <= DATE '2022-12-31'
  EXCEPT
  SELECT o.date
  FROM orders o
  JOIN process p ON p.order_no = o.order_no
  WHERE p.ssn = e.ssn
);

UsageError: Cell magic `%%sql` not found.


3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql
SELECT
    m.month,
    COALESCE(COUNT(o.order_no), 0) AS total
FROM
    (
        SELECT generate_series(1, 12) AS month
    ) m
LEFT JOIN
    orders o ON (EXTRACT(YEAR FROM o.date) = 2022 AND EXTRACT(MONTH FROM o.date) = m.month)
LEFT JOIN
    pay p ON (o.order_no = p.order_no)
GROUP BY
    m.month
ORDER BY
    m.month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql

DROP VIEW IF EXISTS product_sales;

CREATE VIEW product_sales AS
SELECT SKU AS sku, order_no, qty, price*qty AS total_price, EXTRACT(YEAR FROM o.date) AS year, EXTRACT(MONTH FROM o.date) AS month, EXTRACT(DAY FROM o.date) AS day_of_month,
    EXTRACT(DOW FROM o.date) + 1 AS day_of_week, REGEXP_SUBSTR(d.address, '[^,+$ ]+$', 1, 1) AS city
FROM product pd join contains c using (SKU) join orders o using(order_no) join supplier using (SKU) join delivery d using (TIN) join pay using (order_no);


UsageError: Cell magic `%%sql` not found.


## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

A aplicação web tem como ponto de partida um menu principal (home_page.html) que contém 4 botões que levam às listas das 4 entidades presentes na app (Clientes, Produtos, Fornecedores e Encomendas).

Cada um desses botões leva a uma página (products/index.html, por exemplo) onde são listados todos os exemplares da categoria (clientes, produtos, fornecedores ou encomendas) correspondente e onde é possível:
1. Aceder a outra página (products/register.html, por exemplo) para **registar** um novo exemplar se as informações introduzidas para os atributos não violarem qualquer uma das FOREIGN KEYS presentes no esquema relacional dessa categoria;
2. Aceder a uma outra página (products/update.html, por exemplo) onde se encontra **informação mais completa** para cada um dos exemplares e onde é possível **apagar** o exemplar da base de dados, assim como as suas dependências (exceto as encomendas, que não podem ser apagadas); nesta mesma página, na categoria dos produtos, é ainda possível alterar o seu preço ou descrição;

Em particular, no caso das **encomendas**, a página orders/update.html contém ainda botões para outras duas páginas:

- orders/addproduct.html, onde é possível adicionar produtos a uma encomenda que ainda não esteja paga;
- orders/pay.html, onde é possível pagar uma encomenda que ainda não esteja paga, indicando um método de pagamento e desde que o número de cliente introduzido para pagar seja igual ao número de cliente que realizou essa mesma encomenda.

É de notar ainda que para realizar uma nova encomenda é necessário introduzir logo um produto, sendo que os restantes (se existirem) podem ser introduzidos posteriormente.

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
DROP TABLE IF EXISTS dateD;

CREATE TABLE dateD(
    id SERIAL PRIMARY KEY,
    month INTEGER,
    weekday INTEGER,
    day INTEGER);

INSERT INTO dateD (month, day, weekday)
    SELECT EXTRACT(MONTH FROM dd),
    EXTRACT(DAY FROM dd), EXTRACT(DOW FROM dd) + 1
    FROM generate_series(
        '2022-01-01'::timestamp,
        '2022-12-31'::timestamp,
        '1 day'::INTERVAL) dd;

In [ ]:
%%sql
WITH cities(city) as (SELECT DISTINCT city FROM product_sales),
    products_in_2022 as (SELECT SKU FROM product_sales WHERE year = 2022),
    sku_days as (SELECT SKU, month, day, weekday FROM products_in_2022, dateD),
    sku_cities as (SELECT SKU, city FROM products_in_2022, cities),
    sku_cities_days as (SELECT * FROM sku_cities LEFT JOIN sku_days USING (SKU))
SELECT scd.SKU, COALESCE(SUM(qty), 0) as total_qty, COALESCE(SUM(total_price), 0) as total_sales_value, scd.city, scd.month, scd.day, scd.weekday
FROM sku_cities_days as scd LEFT JOIN product_sales as ps ON (scd.SKU = ps.SKU AND scd.month = ps.month AND scd.day = ps.day_of_month
      AND scd.weekday = ps.day_of_week AND scd.city = ps.city)
GROUP BY GROUPING SETS ((scd.SKU), (scd.SKU, scd.city), (scd.SKU, scd.month), (scd.SKU, scd.day), (scd.SKU, scd.weekday))
ORDER BY (scd.SKU, scd.city, scd.month, scd.day, scd.weekday);


2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
WITH products_in_2022 as (SELECT SKU FROM product_sales WHERE year = 2022),
    sku_days as (SELECT SKU, month, weekday FROM products_in_2022, dateD)
SELECT ROUND(COALESCE(AVG(total_price), 0), 2) AS total, sd.month, sd.weekday
FROM sku_days as sd LEFT JOIN product_sales as ps ON (sd.SKU = ps.SKU AND sd.month = ps.month AND sd.weekday = ps.day_of_week)
GROUP BY GROUPING SETS ((), (sd.month), (sd.weekday))
ORDER BY (sd.month, sd.weekday);


## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas:

### 6.1
SELECT order_no
FROM orders
JOIN contains USING (order_no)
JOIN product USING (SKU)
WHERE price > 50 AND
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

Como order_no é primary key de orders, SKU é primary key de product e ambos fazem parte da primary key de contains, então não são necessários mais índices para os JOINs. Por outro lado, a criação de um índice B-tree de chave composta nos atributo date da tabela orders (em primeiro lugar por se tratar de um filtro de igualdade) e price da tabela product (como é um filtro de range, inviabiliza a utilização de um índice hash) agilizaria a execução da consulta.

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains
JOIN product USING (SKU)
WHERE name LIKE ‘A%’
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

Como SKU é primary key de product e faz parte da primary key de contains, então não são necessários mais índices para o JOIN. Por outro lado, como "LIKE 'A%'" consiste num filtro de igualdade tendo em conta o prefixo de name, então seria benéfica a criação de um índice Hash no atributo name da tabela product. Quanto ao agrupamento "GROUP BY order_no", não há benefício em criar um índice em order_no pois como se pretende calcular uma soma é necessário percorrer todas as linhas da tabela resultante do JOIN que permaneçam após a filtragem "LIKE 'A%'" e que digam respeito a cada order_no.